In [16]:
print("Parallel Workflow")

Parallel Workflow


In [17]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict


In [18]:
# Defining state
class Batsmantate(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    strike_rate: float
    runs_in_boundery_persentage: float
    boundery_per_balls_persentage: float
    summary: str



In [ ]:
# defining nodes
def calculate_strike_rate(state: Batsmantate) -> Batsmantate:
    runs = state["runs"]
    balls = state["balls"]
    # calculate strike rate
    sr = (runs/balls)*100

    return {"strike_rate": sr}



def calculate_boundery_persentage(state: Batsmantate):
    runs = state["runs"]
    fours = state["fours"]
    sixes = state["sixes"]

    # calculate boundery_persentage
    score_by_bounderies =  (fours * 4 + sixes * 6)
    boundery_persentage = (score_by_bounderies/runs)*100
    # boundary_percent = (((state['fours'] * 4) + (state['sixes'] * 6))/state['runs'])*100

    return {"runs_in_boundery_persentage": boundery_persentage}


def calculate_boundery_per_balls_persentage(state: Batsmantate):

    
    bpb = ((state['fours'] + state['sixes'])/state["balls"])*100

    return {"boundery_per_balls_persentage": bpb}

def generate_summary(state: Batsmantate):
    summary = f"""
Strike Rate - {state['strike_rate']} \n
Boundary percent - {state['runs_in_boundery_persentage']} \n
Balls per boundary - {state['boundery_per_balls_persentage']} 

"""
    
    return {'summary': summary}





In [20]:
# Defining grapg
graph = StateGraph(Batsmantate)

# Adding nodes and adges
graph.add_node("calculate_strike_rate", calculate_strike_rate)
graph.add_node("calculate_boundery_persentage", calculate_boundery_persentage)
graph.add_node("calculate_boundery_per_balls_persentage", calculate_boundery_per_balls_persentage)
graph.add_node("generate_summary", generate_summary)

graph.add_edge(START, "calculate_strike_rate")
graph.add_edge(START, "calculate_boundery_persentage")
graph.add_edge(START, "calculate_boundery_per_balls_persentage")

graph.add_edge("calculate_strike_rate", "generate_summary")
graph.add_edge("calculate_boundery_persentage", "generate_summary")
graph.add_edge("calculate_boundery_per_balls_persentage", "generate_summary")

graph.add_edge("generate_summary", END)

# compile the graph
workflow = graph.compile()

# exicute the graph
state = { "runs": 100, "balls": 50, "fours": 6, "sixes": 4}

result = workflow.invoke(state)

print(result)

{'runs': 100, 'balls': 50, 'fours': 6, 'sixes': 4, 'strike_rate': 200.0, 'runs_in_boundery_persentage': 48.0, 'boundery_per_balls_persentage': 20.0, 'summary': '\nStrike Rate - 200.0 \n\nBoundary percent - 48.0 \n\nBalls per boundary - 20.0 \n\n'}


In [15]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

## UPSC essay workflow

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated
from dotenv import load_dotenv
import operator

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

class EvaluationSchema(BaseModel):
    feedback:str = Field(..., deprecated="Detailed feedbackfor the essay")
    score: int = Field(description='Score out of 10', ge=0, le=10)


structured_model = model.with_structured_output(EvaluationSchema)



In [3]:
essay = """India in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""

In [4]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt)

EvaluationSchema(feedback="The essay presents a well-structured and comprehensive analysis of India's position in the age of AI.  It effectively highlights both the potential benefits and challenges, showcasing a nuanced understanding of the topic. The writing is clear, concise, and engaging.  However, some specific examples of AI applications in different sectors could be further elaborated to strengthen the impact.  The conclusion effectively summarizes the key points.", score=8)

In [9]:
# defining state
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

# Defining nodes
def evaluate_language(state: UPSCState):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'

    output = structured_model.invoke(prompt)

    return {"language_feedback": output.feedback, "individual_scores": [output.score] }

def evaluate_analysis(state: UPSCState):
    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

def evaluate_thought(state: UPSCState):
    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

def final_avaluation(state: UPSCState):
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'

    overall_feedback = model.invoke(prompt).content

    # calculate the average
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])
    return {"overall_feedback":overall_feedback, "avg_score":avg_score}




In [14]:
# Defining graph
graph = StateGraph(UPSCState)

# adding nodes and edges
graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_thought", evaluate_thought)
graph.add_node("final_avaluation", final_avaluation)

graph.add_edge(START,"evaluate_language" )
graph.add_edge(START,"evaluate_analysis" )
graph.add_edge(START,"evaluate_thought" )

graph.add_edge("evaluate_language", "final_avaluation")
graph.add_edge("evaluate_analysis", "final_avaluation")
graph.add_edge("evaluate_thought", "final_avaluation")

graph.add_edge("final_avaluation", END)

# compile the graph
workfkow = graph.compile()




In [11]:
# exiute the graph
essay = """India in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""

state = {"essay":essay}
result = workfkow.invoke(state)

print(result)

C:\Users\AL HASSAN COMPUTERs\AppData\Local\Temp\ipykernel_12748\501059708.py:29: DeprecationWarning: Detailed feedbackfor the essay
  return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}
C:\Users\AL HASSAN COMPUTERs\AppData\Local\Temp\ipykernel_12748\501059708.py:17: DeprecationWarning: Detailed feedbackfor the essay
  return {"language_feedback": output.feedback, "individual_scores": [output.score] }
C:\Users\AL HASSAN COMPUTERs\AppData\Local\Temp\ipykernel_12748\501059708.py:23: DeprecationWarning: Detailed feedbackfor the essay
  return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}


{'essay': "India in the Age of AI\nAs the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.\n\nIndia's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for 